In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

def scrape_season_stats(year):
    """Scrape NBA player per-game stats for a given season."""
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch data for {year}: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'per_game_stats'})

    # Extract table headers
    headers = [th.text for th in table.find('thead').find_all('th')][1:]  # Skip first column ('Rank')

    # Extract table rows
    rows = []
    for row in table.find('tbody').find_all('tr'):
        if row.find('th', {'scope': 'row'}):  # Skip non-player rows
            rows.append([td.text for td in row.find_all('td')])

    # Create DataFrame
    season_df = pd.DataFrame(rows, columns=headers)
    season_df['Season'] = f"{year-1}-{str(year)[2:]}"  # Add season column
    return season_df

def scrape_multiple_seasons(start_year, end_year):
    """Scrape stats for multiple seasons and save as a CSV."""
    all_data = []

    for year in range(start_year, end_year + 1):
        print(f"Scraping stats for the {year-1}-{str(year)[2:]} season...")
        season_data = scrape_season_stats(year)
        if season_data is not None:
            all_data.append(season_data)

        # Introduce a random delay to avoid being blocked
        delay = random.uniform(5, 15)
        print(f"...waiting {round(delay, 2)} seconds before next request.")
        time.sleep(delay)

    # Combine all data and save
    full_data = pd.concat(all_data, ignore_index=True)
    full_data.to_csv('nba_stats_last_5_years.csv', index=False)
    print("All data scraped and saved to nba_stats_last_5_years.csv!")

# Scrape NBA stats for the last 5 seasons, including 2023-24
scrape_multiple_seasons(2019, 2024)

Scraping stats for the 2018-19 season...
...waiting 12.21 seconds before next request.
Scraping stats for the 2019-20 season...
...waiting 14.88 seconds before next request.
Scraping stats for the 2020-21 season...
...waiting 9.48 seconds before next request.
Scraping stats for the 2021-22 season...
...waiting 14.46 seconds before next request.
Scraping stats for the 2022-23 season...
...waiting 7.98 seconds before next request.
Scraping stats for the 2023-24 season...
...waiting 13.39 seconds before next request.
All data scraped and saved to nba_stats_last_5_years.csv!


In [6]:
df = pd.read_csv('nba_stats_last_5_years.csv')
df.shape

(4296, 31)

In [7]:
lebron_count = df[df['Player'].str.contains('LeBron James', na=False)].shape[0]

print(f"LeBron James appears {lebron_count} times in the dataset.")

LeBron James appears 6 times in the dataset.


In [11]:
df.isnull().sum()

Player       0
Age          6
Team         6
Pos          6
G            6
GS           6
MP           6
FG           6
FGA          6
FG%         36
3P           6
3PA          6
3P%        259
2P           6
2PA          6
2P%         75
eFG%        36
FT           6
FTA          6
FT%        298
ORB          6
DRB          6
TRB          6
AST          6
STL          6
BLK          6
TOV          6
PF           6
PTS          6
Awards    3977
Season       0
dtype: int64

In [19]:
df = df.drop(columns=['Awards'])
df.isnull().sum()

Player    0
Age       0
Team      0
Pos       0
G         0
GS        0
MP        0
FG        0
FGA       0
FG%       0
3P        0
3PA       0
3P%       4
2P        0
2PA       0
2P%       0
eFG%      0
FT        0
FTA       0
FT%       0
ORB       0
DRB       0
TRB       0
AST       0
STL       0
BLK       0
TOV       0
PF        0
PTS       0
Season    0
dtype: int64

In [20]:
df.to_csv('pic16b_nba_stats.csv', index=False)

In [21]:
import sqlite3

In [26]:
conn = sqlite3.connect('nba_stats.db')
df.to_sql('player_stats', conn, if_exists='replace', index=False)
conn.close()

In [27]:
conn = sqlite3.connect('nba_stats.db')
query = "SELECT * FROM player_stats LIMIT 5;"
print(pd.read_sql(query, conn))
conn.close()

                  Player   Age Team Pos     G    GS    MP    FG   FGA    FG%  \
0           James Harden  29.0  HOU  PG  78.0  78.0  36.8  10.8  24.5  0.442   
1            Paul George  28.0  OKC  SF  77.0  77.0  36.9   9.2  21.0  0.438   
2  Giannis Antetokounmpo  24.0  MIL  PF  72.0  72.0  32.8  10.0  17.3  0.578   
3            Joel Embiid  24.0  PHI   C  64.0  64.0  33.7   9.1  18.7  0.484   
4           LeBron James  34.0  LAL  SF  55.0  55.0  35.2  10.1  19.9  0.510   

   ...  ORB   DRB   TRB  AST  STL  BLK  TOV   PF   PTS   Season  
0  ...  0.8   5.8   6.6  7.5  2.0  0.7  5.0  3.1  36.1  2018-19  
1  ...  1.4   6.8   8.2  4.1  2.2  0.4  2.7  2.8  28.0  2018-19  
2  ...  2.2  10.3  12.5  5.9  1.3  1.5  3.7  3.2  27.7  2018-19  
3  ...  2.5  11.1  13.6  3.7  0.7  1.9  3.5  3.3  27.5  2018-19  
4  ...  1.0   7.4   8.5  8.3  1.3  0.6  3.6  1.7  27.4  2018-19  

[5 rows x 30 columns]
